In [6]:
#Exploratory Data Analysis

#Number of unique websites
import json
import matplotlib.pyplot as plt

#read train.csv
#read fids


#find number of unique domains
#
#How varied is the Dataset ?
#Any interesting statistics ?

import os 
import numpy as np
def load(direc):
    examples = []
    fids  = []
    values = []
    for each in os.listdir(direc):        
        if '.npy' in each:
            fids.append(each.split('_')[1][:-4])
            examples.extend(np.load(direc+'/'+each))
            with open(direc+'/'+each[:-4]+'.json', 'r') as infile:
                values.extend(json.load(infile))
    return examples,fids,values


trainexamples, trainfids, trainvalues = load('Contentfeature/train')
trainexamples = np.array(trainexamples)

print 'Number of webpages: %d' %(len(trainfids))
print 'Number of nodes: %d' %(trainexamples.shape[0])
print 'Number of Title Nodes: %d'%(len(filter(lambda x: x[0]==1 , trainexamples)))
print 'Number of Content Nodes: %d' %(len(filter(lambda x: x[0]==2 , trainexamples)))
print 'Number of Date Nodes: %d' %(len(filter(lambda x: x[0]==3 , trainexamples)))


Number of webpages: 269
Number of nodes: 65194
Number of Title Nodes: 266
Number of Content Nodes: 3803
Number of Date Nodes: 204


In [35]:
infile =  open('train/field.csv','r')
websites = {}
for each in infile.readlines():
    domain = each.strip().split(',')[2].split('/')[:3][-1]
    if domain.split('.')[0] == 'www':
        domain = '.'.join(domain.split('.')[1:])
    if each.strip().split(',')[0] in trainfids:
        websites[domain] = websites[domain]+1 if domain in websites else 1
print 'Number of unique domains %d ; Average number of articles per domain %.3f' %(len(websites),sum(e for e in websites.values())*1.0/len(websites))
for each in sorted(websites.keys())[:10]:
    print each, websites[each]
    
res = []
k,v = zip(*d.items())    
for col,key in sorted(zip(v,k)):
    res.append((np.sum(X[:,col]),col,key))
print 'Features that are mostly 0 %s' %(sorted(res)[:10])
print '-*'*10
print 'Features that are always active %s' %(sorted(res)[-10:])

Number of unique domains 239 ; Average number of articles per domain 1.130
11alive.com 1
4029tv.com 1
6abc.com 1
abc-7.com 1
abc.net.au 2
abc13.com 1
abc27.com 1
abcnews.go.com 2
abcnews4.com 1
alaskadispatch.com 1
Features that are mostly 0 [(0.0, 55, u'mspam'), (1.0, 18, u'ancestorTag=form'), (1.0, 36, u'ancestorTag=ssname'), (1.0, 43, u'ancestorTag=u'), (1.0, 47, u'data-font-size=12px'), (1.0, 48, u'data-font-size=14px'), (1.0, 49, u'data-font-size=16px'), (1.0, 67, u'next_tag=comments'), (1.0, 72, u'next_tag=figcaption'), (1.0, 87, u'next_tag=include')]
-*-*-*-*-*-*-*-*-*-*
Features that are always active [(5552.0, 143, u'parent_tag=p'), (9749.0, 35, u'ancestorTag=span'), (9749.0, 146, u'parent_tag=span'), (18520.107491872615, 54, u'functional_word_ratio'), (36235.0, 0, u'ancestorTag=a'), (36235.0, 116, u'parent_tag=a'), (44657.0, 56, u'next_tag=__empty__'), (46503.0, 156, u'previous_tag=__empty__'), (180056.0, 53, u'functional_word_count'), (368296.0, 213, u'word_count')]


In [5]:
#expolaratory analysis
    #does feature space capture 
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
pca = PCA(n_components=2)
trainfeatures = map(lambda x:x[1:] , trainexamples)
trainfeatures1 = normalize(trainfeatures)
X = pca.fit_transform(trainfeatures1)
assert(X.shape[1] == 2)

d = {0:[],1:[],2:[],3:[]}
for i,each in enumerate(trainexamples):
    d[each[0]].append(X[i,:])

x,y = zip(*d[0])
plt.plot(x,y,'bo')
x,y = zip(*d[1])
plt.plot(x,y,'go')
x,y = zip(*d[2])
plt.plot(x,y,'ro')
plt.show()

In [27]:
from sklearn.metrics import f1_score
trainlabels = map(lambda x: x[0]==2, trainexamples)
trainfeatures = map(lambda x: x[1:], trainexamples)
X = np.matrix(trainfeatures)
Y = np.array(trainlabels)
print 'Naieve Base line all Content: %.4f' %(f1_score([1]*len(Y),Y))


Naieve Base line all Content: 0.1102
(65194, 1) (65194,)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [29]:
import numpy as np


with open('Contentfeature/train/featurenames.json') as infile:
    featurenames = json.load(infile)
assert(len(featurenames) == len(trainfeatures[0]))


################ ML methods ######################################


import random

from time import time
from sklearn.svm import SVC
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.grid_search import RandomizedSearchCV
from operator import itemgetter
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint as sp_randint
from sklearn.cross_validation import StratifiedKFold



d = dict(zip(featurenames,range(len(featurenames))))
# print '-----------------'*10
# print 'FeatureNames'
# for each in d.keys():
#     print each
# print '-----------------'*10
###################################################################################################################

# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        if i == 0:
            bestparams = (score.parameters,score.mean_validation_score)
        print("")
    return bestparams

def getf1score(estimator, X, Y):
    #return recall_score(estimator.predict(X),Y)
    return f1_score(estimator.predict(X),Y)



clf1 = SVC(C=1.0)
clf2 = LogisticRegression(penalty='l2', random_state=4,C=1.0)
clf3 = RandomForestClassifier(n_estimators=20)
s3fold = StratifiedKFold(y=Y, n_folds=5,
                                         shuffle=True, random_state=2)
bestparams = []

C = [0.001, 0.001, 0.1, 0.7, 0.5, 0.6, 0.9, 1, 2, 5, 7, 10,50, 70, 100,1000]
gamma = [0.00001, 0.0001,0.001, 0.01, 0.1]
class_weights = [{0:1,1:1},{0:1,1:5},{0:1,1:10},{0:1,1:15},{0:1,1:20}]

param_dist = {"C": C, "gamma":gamma, "class_weight":class_weights}

Xnew = X[:,d['word_count']]



print '#################### SVC  ###############################'
np.random.seed(5)
# # run randomized search
n_iter_search = 20
# random_search = RandomizedSearchCV(clf1, param_distributions=param_dist, scoring=getf1score,
#                                    verbose=2, cv=s3fold )
# start = time()
# random_search.fit(Xnew, Y)
# print("RandomizedSearchCV took %.2f seconds for %d candidates"
#       " parameter settings." % ((time() - start), n_iter_search))
# bestparams.append(report(random_search.grid_scores_))
# print 'Continue ??'
# raw_input()

param_dist = {"C": C,"class_weight": class_weights}
print '#################### LOGISTIC REGRESSION ###############################'
random_search = RandomizedSearchCV(clf2, param_distributions=param_dist, scoring=getf1score,
                                   verbose=2 ,cv=s3fold)

random_search.fit(Xnew, Y)
start = time()
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
bestparams.append(report(random_search.grid_scores_))
print 'Continue ??'
raw_input()



# specify parameters and distributions to sample from
print '#################### RANDOM FOREST ###############################'
param_dist = {"max_depth": [3, None],
              "min_samples_split": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]
             }
random_search = RandomizedSearchCV(clf3, param_distributions=param_dist, scoring=getf1score,
                                   verbose=2 )


start = time()
random_search.fit(Xnew, Y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
bestparams.append(report(random_search.grid_scores_))
print 'BestParams and score'
print bestparams



#################### SVC  ###############################
#################### LOGISTIC REGRESSION ###############################
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] C=1000, class_weight={0: 1, 1: 15} ..............................
[CV] ..................... C=1000, class_weight={0: 1, 1: 15} -   0.1s
[CV] C=1000, class_weight={0: 1, 1: 15} ..............................
[CV] ..................... C=1000, class_weight={0: 1, 1: 15} -   0.1s
[CV] C=1000, class_weight={0: 1, 1: 15} ..............................


[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.1s


[CV] ..................... C=1000, class_weight={0: 1, 1: 15} -   0.1s
[CV] C=1000, class_weight={0: 1, 1: 15} ..............................
[CV] ..................... C=1000, class_weight={0: 1, 1: 15} -   0.1s
[CV] C=1000, class_weight={0: 1, 1: 15} ..............................
[CV] ..................... C=1000, class_weight={0: 1, 1: 15} -   0.1s
[CV] C=50, class_weight={0: 1, 1: 5} .................................
[CV] ........................ C=50, class_weight={0: 1, 1: 5} -   0.1s
[CV] C=50, class_weight={0: 1, 1: 5} .................................
[CV] ........................ C=50, class_weight={0: 1, 1: 5} -   0.1s
[CV] C=50, class_weight={0: 1, 1: 5} .................................
[CV] ........................ C=50, class_weight={0: 1, 1: 5} -   0.1s
[CV] C=50, class_weight={0: 1, 1: 5} .................................
[CV] ........................ C=50, class_weight={0: 1, 1: 5} -   0.1s
[CV] C=50, class_weight={0: 1, 1: 5} .................................
[CV] .

[Parallel(n_jobs=1)]: Done  41 jobs       | elapsed:    3.3s


[CV] .................... C=0.001, class_weight={0: 1, 1: 10} -   0.1s
[CV] C=0.001, class_weight={0: 1, 1: 10} .............................
[CV] .................... C=0.001, class_weight={0: 1, 1: 10} -   0.1s
[CV] C=1000, class_weight={0: 1, 1: 5} ...............................
[CV] ...................... C=1000, class_weight={0: 1, 1: 5} -   0.1s
[CV] C=1000, class_weight={0: 1, 1: 5} ...............................
[CV] ...................... C=1000, class_weight={0: 1, 1: 5} -   0.1s
[CV] C=1000, class_weight={0: 1, 1: 5} ...............................
[CV] ...................... C=1000, class_weight={0: 1, 1: 5} -   0.1s
[CV] C=1000, class_weight={0: 1, 1: 5} ...............................
[CV] ...................... C=1000, class_weight={0: 1, 1: 5} -   0.1s
[CV] C=1000, class_weight={0: 1, 1: 5} ...............................
[CV] ...................... C=1000, class_weight={0: 1, 1: 5} -   0.1s


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    4.1s finished


RandomizedSearchCV took 0.00 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.562 (std: 0.004)
Parameters: {'C': 50, 'class_weight': {0: 1, 1: 10}}

Model with rank: 2
Mean validation score: 0.562 (std: 0.004)
Parameters: {'C': 0.6, 'class_weight': {0: 1, 1: 10}}

Model with rank: 3
Mean validation score: 0.555 (std: 0.005)
Parameters: {'C': 0.001, 'class_weight': {0: 1, 1: 10}}

Continue ??
d
#################### RANDOM FOREST ###############################
Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] min_samples_split=2, bootstrap=False, criterion=gini, max_depth=3, min_samples_leaf=4 
[CV]  min_samples_split=2, bootstrap=False, criterion=gini, max_depth=3, min_samples_leaf=4 -   0.1s
[CV] min_samples_split=2, bootstrap=False, criterion=gini, max_depth=3, min_samples_leaf=4 


[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.1s


[CV]  min_samples_split=2, bootstrap=False, criterion=gini, max_depth=3, min_samples_leaf=4 -   0.1s
[CV] min_samples_split=2, bootstrap=False, criterion=gini, max_depth=3, min_samples_leaf=4 
[CV]  min_samples_split=2, bootstrap=False, criterion=gini, max_depth=3, min_samples_leaf=4 -   0.1s
[CV] min_samples_split=4, bootstrap=True, criterion=gini, max_depth=None, min_samples_leaf=7 
[CV]  min_samples_split=4, bootstrap=True, criterion=gini, max_depth=None, min_samples_leaf=7 -   0.2s
[CV] min_samples_split=4, bootstrap=True, criterion=gini, max_depth=None, min_samples_leaf=7 
[CV]  min_samples_split=4, bootstrap=True, criterion=gini, max_depth=None, min_samples_leaf=7 -   0.2s
[CV] min_samples_split=4, bootstrap=True, criterion=gini, max_depth=None, min_samples_leaf=7 
[CV]  min_samples_split=4, bootstrap=True, criterion=gini, max_depth=None, min_samples_leaf=7 -   0.2s
[CV] min_samples_split=1, bootstrap=False, criterion=entropy, max_depth=3, min_samples_leaf=5 
[CV]  min_samples_sp

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    4.5s finished
